In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import load_dataset
import os

In [23]:
# Cargamos el dataset
dataset_dialogos = load_dataset('json', data_files=r'C:\Users\Lucas\Desktop\ReactAgent\studio\dialogos_procesados.json')
print(dataset_dialogos)
print(dataset_dialogos['train'].features)
print(type(dataset_dialogos['train']['dependencias'][0]))  # No debería ser necesario, ya que 'dependencias' es una cadena

Generating train split: 35 examples [00:00, 404.28 examples/s]

DatasetDict({
    train: Dataset({
        features: ['dialogo', 'titulo', 'entidades', 'temas', 'dependencias'],
        num_rows: 35
    })
})
{'dialogo': Value(dtype='string', id=None), 'titulo': Value(dtype='string', id=None), 'entidades': Value(dtype='string', id=None), 'temas': Value(dtype='string', id=None), 'dependencias': Value(dtype='string', id=None)}
<class 'str'>


In [24]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("¿CUDA disponible?:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU disponible")

def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre   : {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()


¿CUDA disponible?: True
Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Memoria GPU asignada: 1432.41 MB
Memoria GPU reservada: 1536.00 MB
Memoria libre   : 2663.16 MB


In [25]:
# Cargamos el modelo y el tokenizador
model_name = "PlanTL-GOB-ES/gpt2-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name, unk_token="[UNK]")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.to(device)


c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(512, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50265, bias=False)
)

El primer parámetro (50265) es el número total de vocabularios en el conjunto de datos de texto al que se va a aplicar la transformación. En otras palabras, representa la cantidad de palabras únicas que hay en el lenguaje o corpus para el cual se creó este objeto Embedding.

El segundo parámetro (768) es el número de dimensiones del espacio vectorial que utilizará esta transformación. Es decir, representa la cantidad de valores en cada vector que generará para cada palabra o vocablo del lenguaje. Esto permite obtener información sobre las características semánticas y sintácticas de cada palabra

In [32]:
import torch

def tokenizer_dialogo_con_contexto(examples):
    """
    Concatenar las columnas adicionales al texto del diálogo y luego tokenizar.
    """
    dialogo = examples['dialogo']
    titulo = examples['titulo']
    entidades = examples['entidades']
    temas = examples['temas']
    dependencias = examples['dependencias']
    
    # Concatenamos todo el texto para proporcionar contexto adicional
    texto_completo = f"Titulo: {titulo} | Entidades: {entidades} | Temas: {temas} | Dependencias: {dependencias} | Dialogo: {dialogo}"
    
    # Tokenizamos el texto completo con un tamaño máximo controlado
    tokenized_output = tokenizer(
        texto_completo,  # Concatenación de todos los textos
        padding='max_length',  # Rellenar hasta la longitud máxima
        truncation=True,       # Truncar los textos que exceden el tamaño máximo
        max_length=512,        # Establecer un límite máximo de 512 tokens para evitar sobrecarga
        return_tensors='pt'    # Asegurarse de que el retorno sea en formato tensor
    )
    
    # Verificar que se ha generado la cantidad correcta de tokens
    input_ids = tokenized_output['input_ids']
    
    # Asegurarse de que la longitud de input_ids es la esperada
    if input_ids.size(1) != 512:  # Comprobamos si el tamaño es 512
        print(f"Error en tokenización. Longitud esperada 512, pero se obtuvo {input_ids.size(1)}")
        print(f"Texto largo: {texto_completo[:200]}... Tokens generados: {input_ids.size(1)}")
    
    return tokenized_output
    

In [33]:
# Aplicar la tokenización con contexto
tokenized_data = dataset_dialogos.map(tokenizer_dialogo_con_contexto, batched=True)

Map:   0%|          | 0/35 [00:05<?, ? examples/s]


ArrowInvalid: Column 5 named input_ids expected length 35 but got length 1

In [ ]:
# Verificar que las columnas ahora son tensores
print(type(tokenized_data['train']['dialogo'][0]))  # Debería ser un tensor de PyTorch
print(type(tokenized_data['train']['attention_mask'][0]))  # También un tensor de PyTorch

<class 'list'>
<class 'list'>


In [ ]:
# Verificar el tipo de datos dentro del dataset tokenizado
print(type(tokenized_data['train']['dialogo'][0]))  # Debería ser un tensor de PyTorch
print(type(tokenized_data['train']['attention_mask'][0]))  # También un tensor de PyTorch

# Comprobar si se convierte correctamente a tensor
print(tokenized_data['train']['dialogo'][0]) 

<class 'list'>
<class 'list'>
[2, 26736, 80, 275, 253, 253, 26736, 80, 2558, 989, 23544, 66, 6972, 383, 415, 45386, 313, 1498, 41096, 529, 8296, 24844, 839, 342, 23758, 367, 387, 18628, 369, 4489, 634, 12560, 1857, 468, 66, 361, 6886, 66, 8605, 772, 389, 1066, 415, 45386, 442, 341, 313, 1650, 8209, 334, 355, 5238, 313, 2206, 4623, 66, 341, 361, 8972, 334, 6523, 2942, 68, 415, 45386, 347, 355, 2165, 13551, 394, 43556, 414, 320, 332, 10576, 66, 342, 36865, 840, 341, 2702, 719, 13976, 7341, 68, 16260, 13038, 338, 334, 344, 764, 5159, 313, 403, 2426, 313, 1609, 68, 1893, 11383, 7635, 5493, 344, 6487, 313, 499, 2822, 13892, 81, 347, 11407, 1429, 3658, 5493, 320, 15560, 6468, 68, 547, 11407, 341, 361, 683, 22150, 344, 3057, 66, 361, 683, 2198, 318, 387, 6978, 342, 364, 15557, 66, 342, 509, 347, 2721, 66, 361, 683, 1394, 12436, 796, 1764, 389, 6016, 68, 1893, 347, 737, 8243, 44749, 1609, 509, 514, 8842, 347, 13642, 66, 623, 6077, 845, 442, 9467, 66, 344, 253, 253, 26736, 80, 275, 253, 253, 26

In [ ]:
# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./checkpoints",             
    evaluation_strategy="epoch",            # Evaluación después de cada época
    save_strategy="epoch",                 # Guardar checkpoints después de cada época
    per_device_train_batch_size=16,        # Tamaño del batch por dispositivo (16 en este caso)
    num_train_epochs=20,                   # Número de épocas para el entrenamiento
    logging_dir="./logs",                  # Directorio para guardar los logs
    logging_steps=10,                      # Frecuencia de los logs
    save_total_limit=2,                    # Limitar el número de checkpoints guardados
    report_to="none",                      # No reportar a plataformas como TensorBoard
    remove_unused_columns=False,           # No eliminar columnas no utilizadas
    gradient_accumulation_steps=2,         # Acumulación de gradientes cada 2 pasos
    learning_rate=1e-5,                    # Tasa de aprendizaje ajustada
)

In [ ]:
# Definir la función de evaluación
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == labels).float().mean()
    return {'accuracy': accuracy.item()}